In [1]:
# 동적 selenium 정적 bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
# 드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [8]:
# 검색어 설정
keyword = "파이썬"
search_url = f"https://search.naver.com/search.naver?where=news&query={keyword}"

# 검색 페이지 접속
driver.get(search_url)
time.sleep(2)

# 뉴스 링크 수집
news_links = []
links = driver.find_elements(By.CSS_SELECTOR, "a.info")

# 네이버 뉴스 링크만 수집 (3개)
for link in links:
    if len(news_links) >= 3:  # 원하는 기사 수로 수정 가능
        break
    href = link.get_attribute('href')
    if href and 'news.naver.com' in href:
        news_links.append(href)

print(f"수집된 뉴스 링크 수: {len(news_links)}")

수집된 뉴스 링크 수: 3


In [9]:
new_data =[]

for url in news_links:
    driver.get(url)
    time.sleep(2)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        title = soup.select_one('#title_area span').text.strip()
        
        content = soup.select_one('#dic_area').text.strip()
        new_data.append({
            'title':title,
            'content':content,
            'url':url
        })
        
        print(f'제목:{title}')
        print("-" * 50)
    except Exception as e:
        print(f'Error with {url}:  {e}')
driver.quit()

제목:‘파이썬과 챗GPT의 만남’ 코딩밸리, ‘ChatGPT로 파이썬 제대로 활용하기’ 강의 업데이트
--------------------------------------------------
제목:과기정보연, 대전교육청 협력 학습데이터 분석 교육 프로그램 운영
--------------------------------------------------
제목:[단독]구광모 힘주는 AI…LG전자, 공공기관에 'AI 노하우' 공유
--------------------------------------------------


In [12]:
df = pd.DataFrame(new_data)

print('\n==수집된 뉴스 목록===')
for i, row in df.iterrows():
    print(f'\n[기사 {i+1}]')
    print(f"제목: {row['title']}")
    print(f"url: {row['url']}")
    print('-'*50)
    
# Excel 파일로 저장
df.to_excel(f'{keyword}_news.xlsx', index=False)
print(f"\n결과가 {keyword}_news.xlsx 파일로 저장되었습니다.")


==수집된 뉴스 목록===

[기사 1]
제목: ‘파이썬과 챗GPT의 만남’ 코딩밸리, ‘ChatGPT로 파이썬 제대로 활용하기’ 강의 업데이트
url: https://n.news.naver.com/mnews/article/081/0003509796?sid=101
--------------------------------------------------

[기사 2]
제목: 과기정보연, 대전교육청 협력 학습데이터 분석 교육 프로그램 운영
url: https://n.news.naver.com/mnews/article/421/0008012476?sid=102
--------------------------------------------------

[기사 3]
제목: [단독]구광모 힘주는 AI…LG전자, 공공기관에 'AI 노하우' 공유
url: https://n.news.naver.com/mnews/article/018/0005913701?sid=101
--------------------------------------------------

결과가 파이썬_news.xlsx 파일로 저장되었습니다.
